[View in Colaboratory](https://colab.research.google.com/github/jsxyhelu/DateSets/blob/master/DataAugmentation_eg2_%E6%95%88%E8%83%BD%E9%AA%8C%E8%AF%81.ipynb)

In [1]:
from keras.datasets import cifar10
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import generic_utils

(x_train, y_train),(x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

def preprocess_data(x):
    x /= 255
    x -= 0.5
    x *= 2
    return x

# 预处理
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

# one-hot encoding
n_classes = 10
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

# 取 20% 的训练数据
x_train_part = x_train[:10000]
y_train_part = y_train[:10000]

print(x_train_part.shape, y_train_part.shape)

# 建立一个简单的卷积神经网络,序贯结构
def build_model():
    model = Sequential()

    model.add(Conv2D(64, (3,3), input_shape=(32,32,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(scale=False, center=False))

    model.add(Conv2D(32, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization(scale=False, center=False))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    return model

# 训练参数
batch_size = 128
epochs = 5
#epochs = 2
#cifar-10 20%数据,训练结果
model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_part, y_train_part, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.1)

loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Loss: ', loss)
print('Accuracy: ', acc)

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(10000, 32, 32, 3) (10000, 10)
Train on 9000 samples, validate on 1000 samples
Epoch 1/5
9000/9000 [==============================] - 5s 563us/step - loss: 1.8281 - acc: 0.3979 - val_loss: 1.3963 - val_acc: 0.5050
Epoch 2/5
9000/9000 [==============================] - 3s 352us/step - loss: 1.2127 - acc: 0.5729 - val_loss: 1.3446 - val_acc: 0.5230
Epoch 3/5
9000/9000 [==============================] - 3s 354us/step - loss: 0.9205 - acc: 0.6732 - val_loss: 1.3737 - val_acc: 0.5260
Epoch 4/5
9000/9000 [==============================] - 3s 353us/step - loss: 0.7135 - acc: 0.7488 - val_loss: 1.2685 - val_acc: 0.5620
Epoch 5/5
10000/10000 [==============================] - 2s 158us/step
Loss:  1.2000079774856567
Accuracy:  0.5939


In [2]:
#cifar-10 20%数据 + Data Augmentation.训练结果
# 设置生成参数
img_generator = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2
    )
model_2 = build_model()
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

自定义ImageDataGenerator

In [3]:
# Data Augmentation
for e in range(epochs):
    print('Epoch', e)
    print('Training...')
    progbar = generic_utils.Progbar(x_train_part.shape[0])
    batches = 0

    for x_batch, y_batch in img_generator.flow(x_train_part, y_train_part, batch_size=batch_size, shuffle=True):
        loss,train_acc = model_2.train_on_batch(x_batch, y_batch)
        batches += x_batch.shape[0]
        if batches > x_train_part.shape[0]:
            break
        progbar.add(x_batch.shape[0], values=[('train loss', loss),('train acc', train_acc)])
loss, acc = model_2.evaluate(x_test, y_test, batch_size=32)
print('Loss: ', loss)
print('Accuracy: ', acc)

Epoch 0
Training...
10000/10000 [==============================] - 8s 846us/step - train loss: 2.0566 - train acc: 0.3071
Epoch 1
Training...
10000/10000 [==============================] - 7s 719us/step - train loss: 1.7397 - train acc: 0.3799
Epoch 2
Training...
10000/10000 [==============================] - 7s 720us/step - train loss: 1.6284 - train acc: 0.4170 6s - train loss: 1.6095 - train  - ETA: 6s - train loss: 1.6539 - ET
Epoch 3
Training...
10000/10000 [==============================] - 7s 716us/step - train loss: 1.5622 - train acc: 0.4396 5s - train loss: 1.5611 -  - ETA
Epoch 4
Training...
10000/10000 [==============================] - 2s 218us/step
Loss:  1.4424572896957397
Accuracy:  0.5056


In [4]:
###最后，我尝试采用文档中提示方法
img_generator.fit(x_train_part)

In [ ]:
# fits the model_2 on batches with real-time data augmentation:
# ImageDataGenerator.flow 将会返回一个生成器，这个生成器用来扩充数据，每次都会产生batch_size个样本。
model_2.fit_generator(img_generator.flow(x_train_part, y_train_part, batch_size=batch_size),
                    steps_per_epoch=len(x_train_part)//10, epochs=epochs)

Epoch 1/5
1000/1000 [==============================] - 52s 52ms/step - loss: 1.3042 - acc: 0.5303
Epoch 2/5
1000/1000 [==============================] - 53s 53ms/step - loss: 1.1199 - acc: 0.5992
Epoch 3/5
1000/1000 [==============================] - 53s 53ms/step - loss: 1.0211 - acc: 0.6369
Epoch 4/5
1000/1000 [==============================] - 53s 53ms/step - loss: 0.9599 - acc: 0.6588
Epoch 5/5
 587/1000 [================>.............] - ETA: 21s - loss: 0.9255 - acc: 0.6715

In [ ]:
# Score trained model.
scores = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])